In [1]:
## Importing libraries needed.

from fuzzywuzzy import fuzz, process
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import numpy as np
from itertools import zip_longest
import re

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Prior to starting this process, download a salesforce CSV report of the Accounts objects. Save this file as a CSV UTF-8

In [2]:
## Read csv report from salesforce and store as acct object.

acct=pd.read_csv('/kaggle/input/example-data/accounts_example.csv')

In [3]:
## View head of dataframe to make sure it was imported correctly. 

acct.head()

,Account Owner,Account Name,Billing State/Province (text only)
0,John Smith,Example 1 - Lowkey Enterprises,Iowa
1,John Smith,Example 1 - Lowkey,Iowa
2,John Smith,"Example 1 - Lowkey Enterprises, Inc.",Iowa
3,Jane Smith,Example 2 - Rational Workings,Minnesota
4,Jane Smith,Example 2 - Ratoinal Wrokings,Minnesota


In [4]:
## View shape of original dataframe for reference throughout job.

acct.shape

(6, 3)

In [5]:
## Duplicate Account Name column for manipulation. View head to make sure new column was added successfully.

acct['co']=acct.loc[:,'Account Name']
acct.head()

,Account Owner,Account Name,Billing State/Province (text only),co
0,John Smith,Example 1 - Lowkey Enterprises,Iowa,Example 1 - Lowkey Enterprises
1,John Smith,Example 1 - Lowkey,Iowa,Example 1 - Lowkey
2,John Smith,"Example 1 - Lowkey Enterprises, Inc.",Iowa,"Example 1 - Lowkey Enterprises, Inc."
3,Jane Smith,Example 2 - Rational Workings,Minnesota,Example 2 - Rational Workings
4,Jane Smith,Example 2 - Ratoinal Wrokings,Minnesota,Example 2 - Ratoinal Wrokings


In [6]:
## Remove all uppercase letters from 'co' column. View head to make sure change was applied properly.

acct['co']=acct['co'].apply(str.lower)
acct.head()

,Account Owner,Account Name,Billing State/Province (text only),co
0,John Smith,Example 1 - Lowkey Enterprises,Iowa,example 1 - lowkey enterprises
1,John Smith,Example 1 - Lowkey,Iowa,example 1 - lowkey
2,John Smith,"Example 1 - Lowkey Enterprises, Inc.",Iowa,"example 1 - lowkey enterprises, inc."
3,Jane Smith,Example 2 - Rational Workings,Minnesota,example 2 - rational workings
4,Jane Smith,Example 2 - Ratoinal Wrokings,Minnesota,example 2 - ratoinal wrokings


In [7]:
## Removing all punctuation and common words from 'co' column prior to performing
## fuzzy match comparison. These common words conflict with the fuzzy match algorithm
## and therefore must be removed. 

unwanted = '[\.,\-\/&]'
substitutions = {' inc': '', ' incorporated': '', ' llc': '', ' pllc': '',
                 ' corp': '', ' corporation': '', ' group': '', ' logistics': '',
                 'logistics ': '', ' insurance': '', ' associates': '', ' company': '',
                 ' service': '', ' services': '', ' enterprises': '', ' enterprise': '',
                 ' solutions': '', ' contractors': '', ' contractor': '', 
                 ' contracting': '', ' manufacturing': '', ' mechanical': '',
                ' delivery': '', ' companies': '', ' electric': '', 
                ' industries': '', ' construction': '', ' transport': '',
                ' engineering': '', ' constructors': '', ' holdings': '',
                ' international': '', ' transportation': '', ' management': '',
                ' distribution': ''}

pattern = re.compile('|'.join(substitutions.keys()))

def clean_company_name(name):
    name = name.lower().strip()
    name = re.sub(unwanted, ' ', name)
    name = pattern.sub(lambda x: substitutions[x.group()], name)
    name = name.replace(' ', '')
    return name.strip()

acct['co'] = acct['co'].apply(clean_company_name)


In [8]:
## Removing instances of more than one space from entries in the 'co' column.

acct['co']=acct['co'].str.replace(r'\s{2,}', '', regex=True)


In [9]:
## Creating a subset with exact duplicates prior to searching for fuzzy matches. These exact duplicates
## are stored in a dataframe called exact_dups.

duplicate_indices = acct[acct['co'].duplicated(keep=False)].index.tolist()
exact_dups = acct.loc[duplicate_indices]

exact_dups.head()

,Account Owner,Account Name,Billing State/Province (text only),co
0,John Smith,Example 1 - Lowkey Enterprises,Iowa,example1lowkey
1,John Smith,Example 1 - Lowkey,Iowa,example1lowkey
2,John Smith,"Example 1 - Lowkey Enterprises, Inc.",Iowa,example1lowkey


In [10]:
## Viewing shape of exact duplicates. This will tell us how many exact duplicates were found.

exact_dups.shape

(3, 4)

In [11]:
## Removing exact duplicates from the original dataframe before we proceed to search for fuzzy matches.

duplicate_indices = acct[acct['co'].duplicated(keep=False)].index.tolist()
non_dup_subset = acct.loc[~acct.index.isin(duplicate_indices)]
non_dup_subset.head()


,Account Owner,Account Name,Billing State/Province (text only),co
3,Jane Smith,Example 2 - Rational Workings,Minnesota,example2rationalworkings
4,Jane Smith,Example 2 - Ratoinal Wrokings,Minnesota,example2ratoinalwrokings
5,Jane Smith,"Example 2 - Rational Workings, LTD",Minnesota,example2rationalworkingsltd


In [12]:
## Viewing shape of the non-duplicates to see how many rows we have left.

non_dup_subset.shape

(3, 4)

In [13]:
## Performing fuzzy matching. The number `90` in the line `if r[1] >= 90:` can be adjusted. This sets the tolerance
## for the fuzzy matching. This algorithm rates fuzzy matches on a scale of 0 to 100. So this algorithm will take
## anything that scores 90 or more on the fuzzy scale and store it in a fuzzy_dups dataframe. At the end of this
## process, if you find that there are too many non-duplicates in the dataframe, you can rerun the job with a
## higher tolerance and vice-versa if there are too many duplicates.

import pandas as pd
from fuzzywuzzy import fuzz, process

fuzzy_dups = []

for i, row in non_dup_subset.iterrows():
    choices = non_dup_subset['co'].drop(i)
    ratios = process.extract(row['co'], choices, scorer=fuzz.token_sort_ratio)
    for r in ratios:
        if r[1] >= 90:
            fuzzy_dups.append({'co1': row['co'], 'co2': r[0], 'score': r[1]})

fuzzy_dups = pd.concat([pd.DataFrame(x, index=[0]) for x in fuzzy_dups], ignore_index=True)


In [14]:
## Viewing shape of fuzzy duplicates dataframe to see how many fuzzy duplicates were identified. This is represented
## as `(rows, columns)` in the output.

fuzzy_dups.shape

(4, 3)

In [15]:
## This code block creates a new dataframe called fuzzy_dups_2 that will be your fuzzy duplicates ending dataframe. 
## It will be called fuzzy_dups_2

# Get all unique values of co1 and co2 in fuzzy_dups
fuzzy_dups_values = pd.concat([fuzzy_dups['co1'], fuzzy_dups['co2']]).unique()

# Filter contacts where the co value is in fuzzy_dups_values
fuzzy_dups_2 = acct[acct['co'].isin(fuzzy_dups_values)]

fuzzy_dups_2.head()

,Account Owner,Account Name,Billing State/Province (text only),co
3,Jane Smith,Example 2 - Rational Workings,Minnesota,example2rationalworkings
4,Jane Smith,Example 2 - Ratoinal Wrokings,Minnesota,example2ratoinalwrokings
5,Jane Smith,"Example 2 - Rational Workings, LTD",Minnesota,example2rationalworkingsltd


In [16]:
## The final shape view of the fuzzy_dups_2 dataframe will show how many entries you're working with. 

fuzzy_dups_2.shape

(3, 4)

In [17]:
## Exporting the exact dups and fuzzy dups to csv files.

fuzzy_dups_2.to_csv('fuzzy_dups_acct_3.csv',index=True)
exact_dups.to_csv('exact_dups_acct.csv',index=True)

# Regarding the exported csv files:
## The exact dups csv file is good to go. The fuzzy dups file will need to be gone through with a human eye to
## search for what is actually a fuzzy match and what is not. Open the csv file in Excel and sort by the 'co'
## column. Then go through visually and look for fuzzy matches in the 'Account Name' column of the dataframe. 
## Delete anything that is not a fuzzy match. The leftovers will be all of your fuzzy matches. You can then combine
## the two dataframes into one final duplicates report. 